<a href="https://colab.research.google.com/github/kgreed4/BottleBuddy/blob/dombuford/DL3_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#grab data
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d zynicide/wine-reviews


Saving kaggle.json to kaggle.json
kaggle.json
 98% 50.0M/50.9M [00:00<00:00, 151MB/s] 
100% 50.9M/50.9M [00:00<00:00, 127MB/s]


In [ ]:
!unzip wine-reviews.zip

Archive:  wine-reviews.zip
  inflating: winemag-data-130k-v2.csv  
  inflating: winemag-data-130k-v2.json  
  inflating: winemag-data_first150k.csv  


In [ ]:
import pandas as pd
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
df = pd.read_csv('winemag-data-130k-v2.csv')
df.drop(columns = ['Unnamed: 0','description', 'region_1', 'region_2', 'taster_twitter_handle'], inplace = True)

In [ ]:
#handle missing values
df_drop_na = df.dropna(subset = ['variety', 'province', 'country'])
df_drop_na['designation']= df_drop_na['designation'].fillna('no designation')
df_drop_na['price'] = df_drop_na.groupby(['winery', 'variety'])['price'].transform(lambda x: x.fillna(x.mean()))
df_drop_na['price'] = df_drop_na.groupby(['variety'])['price'].transform(lambda x: x.fillna(x.mean()))
df_drop_na = df_drop_na.dropna(subset = ['price'])

<ipython-input-5-d4472aa6e8be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop_na['designation']= df_drop_na['designation'].fillna('no designation')
<ipython-input-5-d4472aa6e8be>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop_na['price'] = df_drop_na.groupby(['winery', 'variety'])['price'].transform(lambda x: x.fillna(x.mean()))
<ipython-input-5-d4472aa6e8be>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [ ]:
value_counts = df_drop_na['winery'].value_counts()
values_to_drop = value_counts[value_counts<5].index
df_drop_na = df_drop_na[~df_drop_na['winery'].isin(values_to_drop)]

In [ ]:
value_counts= df_drop_na['designation'].value_counts()
values_to_drop = value_counts[value_counts<50].index
df_drop_na = df_drop_na[~df_drop_na['designation'].isin(values_to_drop)]


In [ ]:
value_counts = df_drop_na['variety'].value_counts()
values_to_drop = value_counts[value_counts<100].index
df_drop_na = df_drop_na[~df_drop_na['variety'].isin(values_to_drop)]

In [ ]:
value_counts = df_drop_na['province'].value_counts()
values_to_drop = value_counts[value_counts<100].index
df_drop_na = df_drop_na[~df_drop_na['province'].isin(values_to_drop)]

In [ ]:
df_drop_na.shape

(33122, 9)

In [ ]:
#df_final = df_drop_na.drop(columns = ['price'])
df_final = df_drop_na.reset_index(drop=True)

In [ ]:
data = pd.get_dummies(df_final, columns=['country', 'designation', 'province', 'variety', 'winery' ])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


In [ ]:
scaler = StandardScaler()
data_withoutName = data.drop(columns = ['title', 'taster_name', 'points'], axis=1)
wine_names = data['title']
scaled_data = scaler.fit_transform(data_withoutName)

k = 6  # Number of neighbors to consider
knn_model = NearestNeighbors(n_neighbors=k, metric='euclidean')  # You can use different metrics like 'cosine' for similarity
knn_model.fit(scaled_data)








NearestNeighbors(metric='euclidean', n_neighbors=6)

In [ ]:
tasters = data['taster_name'].unique().tolist()
final = []
for name in tasters:
  tasters_differences = []
  check =  data[data['taster_name']==name]
  check.sort_values(by='points', ascending=False, inplace = True)

  voters_top_10 = check.head(10)
  point_values = voters_top_10['points'].tolist()
  indexes = voters_top_10.index.tolist()
  for i in range(len(indexes)):
    ranked_differences = []
    index_value = indexes[i]
    reference_index = index_value # Adjust as needed
    reference_wine = scaled_data[reference_index].reshape(1, -1)  # Reshape to match the input format expected by kneighbors

    distances, indices = knn_model.kneighbors(reference_wine)

    nearest_neighbor_indices = indices[0]  # Indices of nearest neighbors
    nearest_neighbor_names = wine_names.iloc[nearest_neighbor_indices]
    nearest_neighbor_names = nearest_neighbor_names[1:]

    #get the points
    for nearest_neighbor_name in nearest_neighbor_names:
      if nearest_neighbor_name in check['title'].values:
        # Get the row where the name is present in the 'title' column
        row = check[check['title'] == nearest_neighbor_name]
        # Get the points column value from the row
        difference = abs(point_values[i]-row['points'].values[0])
        ranked_differences.append(difference)
    threshold_sum = 0
    for x in ranked_differences:
      if x<=5:
        threshold_sum+=1
    if len(ranked_differences)!=0:
      tasters_differences.append(threshold_sum/len(ranked_differences))
  final.append(tasters_differences)





<ipython-input-26-ff0663898dae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.sort_values(by='points', ascending=False, inplace = True)
<ipython-input-26-ff0663898dae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.sort_values(by='points', ascending=False, inplace = True)
<ipython-input-26-ff0663898dae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.sort_values(by='points', ascending=False, inplace = True)
<ipy

In [ ]:
final

[[1.0, 1.0, 1.0, 1.0, 1.0, 0.6, 0.8, 1.0, 0.6666666666666666, 1.0],
 [0.6, 0.5, 1.0, 0.6, 1.0, 0.8, 0.6, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.3333333333333333, 0.6, 1.0, 1.0, 1.0, 1.0, 0.4, 1.0, 1.0],
 [],
 [0.4, 1.0, 0.8, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 0.8, 0.8, 0.6666666666666666, 0.8, 0.5, 1.0, 1.0],
 [0.25, 0.5, 0.2, 0.4, 0.8, 0.5, 1.0, 0.6, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 0.75, 0.75, 0.4, 0.8, 1.0, 0.5, 1.0, 0.5, 1.0],
 [1.0, 0.75, 1.0, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 []]

In [ ]:
data['taster_name'].value_counts()

taster_name
Roger Voss            5777
Paul Gregutt          3333
Virginie Boone        3269
Michael Schachner     2583
Kerin O’Keefe         2312
Matt Kettmann         1900
Jim Gordon            1546
Sean P. Sullivan      1463
Anna Lee C. Iijima    1180
Joe Czerwinski         766
Anne Krebiehl MW       516
Susan Kostrzewa        126
Lauren Buzzeo          119
Alexander Peartree      85
Carrie Dykes            50
Christina Pickard        3
Name: count, dtype: int64